<a href="https://colab.research.google.com/github/BohdanPetryshyn/code-llama-fim-fine-tuning/blob/main/code_llama_fim_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Code Llama FIM fine-tuning

If you found a problem with this notebook, please report it in the original GitHub [repo](https://github.com/BohdanPetryshyn/code-llama-fim-fine-tuning?tab=readme-ov-file) as an issue.

## Install dependenices

In [1]:
!git clone https://github.com/BohdanPetryshyn/code-llama-fim-fine-tuning.git repo

Cloning into 'repo'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 28 (delta 11), reused 27 (delta 10), pack-reused 0
Receiving objects: 100% (28/28), 16.84 KiB | 16.84 MiB/s, done.
Resolving deltas: 100% (11/11), done.


In [2]:
!pip install ninja
!ninja --version

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 8.1 MB/s eta 0:00:00
1.11.1.git.kitware.jobserver-1


In [3]:
%cd repo
!pip install -r requirements.txt

/content/repo
  Cloning https://github.com/unslothai/unsloth.git (to revision 27fa021a7bb959a53667dd4e7cdb9598c207aa0d) to /tmp/pip-install-wztqjbse/unsloth_9c722ab323924c2bb9712de818327d4b
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-wztqjbse/unsloth_9c722ab323924c2bb9712de818327d4b
  Running command git rev-parse -q --verify 'sha^27fa021a7bb959a53667dd4e7cdb9598c207aa0d'
  Running command git fetch -q https://github.com/unslothai/unsloth.git 27fa021a7bb959a53667dd4e7cdb9598c207aa0d
  Running command git checkout -q 27fa021a7bb959a53667dd4e7cdb9598c207aa0d
  Resolved https://github.com/unslothai/unsloth.git to commit 27fa021a7bb959a53667dd4e7cdb9598c207aa0d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.4/354.4 kB 8.9 MB/s eta 0:00:00
     

## Login to Weights and Biases and Hugging Face

In [4]:
!wandb login --relogin

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [5]:
from huggingface_hub import notebook_login

notebook_login()

## Train the model

This script starts the training process. The fine-tuned model will be saved to your Hugging Face account. The progress can be tracked in real time in Weights and Biases.

In [6]:
# Replace (at least) the following:
#   - dataset_name - your own dataset
#   - output_dir   - output model id in you Hugging Face account
!python train.py \
  --model_name_or_path "codellama/CodeLlama-7b-hf" \
  --dataset_name "BohdanPetryshyn/openapi-completion-refined" \
  --splits "train" \
  --max_seq_len 2048 \
  --max_steps 100 \
  --save_steps 100 \
  --eval_steps 100 \
  --logging_steps 5 \
  --log_level "info" \
  --logging_strategy "steps" \
  --evaluation_strategy "steps" \
  --save_strategy "steps" \
  --push_to_hub \
  --hub_private_repo False \
  --hub_strategy "every_save" \
  --bf16 True \
  --learning_rate 2e-4 \
  --lr_scheduler_type "cosine" \
  --weight_decay 0.1 \
  --warmup_ratio 0.05 \
  --max_grad_norm 1.0 \
  --output_dir "tmp-codellama-7b-openapi-completion-ctx-lvl-fim-05-spm-2048" \
  --per_device_train_batch_size 4 \
  --per_device_eval_batch_size 4 \
  --gradient_accumulation_steps 8 \
  --gradient_checkpointing True \
  --use_reentrant True \
  --dataset_text_field "content" \
  --test_size 0.1 \
  --fim_rate 0.9 \
  --fim_spm_rate 0.5 \
  --use_peft_lora True \
  --lora_r 32 \
  --lora_alpha 64 \
  --lora_dropout 0.1 \
  --lora_target_modules "all-linear" \
  --use_4bit_quantization True \
  --use_nested_quant True \
  --bnb_4bit_compute_dtype "bfloat16" \
  --use_flash_attn True

2024-06-15 09:00:43.075177: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-15 09:00:43.127223: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-15 09:00:43.127275: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-15 09:00:43.128861: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-15 09:00:43.137416: I tensorflow/core/platform/cpu_feature_guar